In [1]:
import bpy
import bmesh
import mathutils as bm

/run/user/1000/gvfs/ non-existent directory


In [2]:
class box:
    def __init__(self):
        self.min = None
        self.max = None

    def add(self, p):
        if self.min is None or self.max is None:
            self.min = p
            self.max = p
            return
        self.min = bm.Vector((min(x, y) for x,y in zip(self.min, p)))
        self.max = bm.Vector((max(x, y) for x,y in zip(self.max, p)))
        return self
    def size(self):
        return self.max - self.min
    def radius(self):
        return self.size().length
    def center(self): 
        return (self.max + self.min) * 0.5

    def create_box(self, mesh, margin=0):
        m = bm.Matrix.LocRotScale(self.center(), None, self.size() + bm.Vector.Fill(3, 2 * margin))
        return bmesh.ops.create_cube(mesh, size=1, matrix=m)

    @staticmethod
    def bbox(mesh):
        b = box()
        for i in mesh.verts[:]:
            b.add(i.co)
        return b

In [14]:
import math
class meshutils:
    eps = 1e-5
    @staticmethod
    def classify_point(plane, point):
        d = (plane.xyz @ point) - plane.w
        if d > meshutils.eps:
            return 0x1 # front
        if d < -meshutils.eps:
            return 0x2 # back
        return 0x0 # in plane

    @staticmethod
    def point_positive(plane, point):
        return ((plane.xyz @ point) - plane.w) > 0
        
    @staticmethod
    def classify_face(plane, face):
        res = 0
        for v in face.verts[:]:
            c = meshutils.classify_point(plane, v.co)
            res |= c
            if res == 0x3: # crossing
                break
        return res

    @staticmethod
    def get_plane(face):
        return bm.Vector((*face.normal, face.normal @ face.verts[0].co))

    @staticmethod
    def slice_mesh(mesh, plane, clean_outer=True):
        bmesh.ops.bisect_plane(mesh, geom=mesh.faces[:] + mesh.edges[:] + mesh.verts[:], 
            plane_co=plane.xyz * plane.w, plane_no=plane.xyz, clear_inner=not clean_outer, clean_outer=clean_outer)
        bmesh.ops.holes_fill(mesh, edges=mesh.edges, sides=len(mesh.edges))
        bmesh.ops.delete(mesh, geom=[x for x in mesh.faces if x.calc_area() < meshutils.eps], context='FACES')
        for face in mesh.faces:
            bmesh.ops.remove_doubles(mesh, verts=face.verts[:], dist=meshutils.eps)
    @staticmethod
    def split(mesh, plane):
        # suboptimal (probably need to split before clean)
        front = mesh.copy()
        return (meshutils.slice_mesh(front, plane, False), meshutils.slice_mesh(mesh, plane, True))


In [7]:
bm.geometry.points_in_planes([
    bm.Vector(( 1, 0, 0, 1)),
    bm.Vector(( 0, 1, 0, 1)),
    bm.Vector(( 0, 0, 1, 1)),
])

([Vector((-1.0, -1.0, -1.0))], [0, 1, 2])

In [ ]:
from enum import IntFlag
import qsl

class bsp_face_build_flags(IntFlag):
    hint = qsl.Q_SURF_HINT # preferable splitter
    areaportal = qsl.Q_CONT_AREAPORTAL # separates areas
    translucent = qsl.Q_CONT_TRANSLUCENT # has geometry below (doesn't split areas)
    nodraw = qsl.Q_SURF_NODRAW # doesn't present as face
class bsp_face_flags(IntFlag):
    slick = qsl.Q_SURF_SLICK
    noimpact = qsl.Q_SURF_NOIMPACT
    nomarks = qsl.Q_SURF_NOMARKS
    nodamage = qsl.Q_SURF_NODAMAGE
    noob = qsl.Q_SURF_NOOB
    metalsteps = qsl.Q_SURF_METALSTEPS
    flesh = qsl.Q_SURF_FLESH
    nosteps = qsl.Q_SURF_NOSTEPS
    dust = qsl.Q_SURF_DUST
class bsp_content_flags(IntFlag): # affect brushes only
    trigger = qsl.Q_CONT_TRIGGER
    playerclip = qsl.Q_CONT_PLAYERCLIP
    water = qsl.Q_CONT_WATER
    slime = qsl.Q_CONT_SLIME
    lava = qsl.Q_CONT_LAVA
    nodrop = qsl.Q_CONT_NODROP
    fog = qsl.Q_CONT_FOG

class _bsp_sub:
    @classmethod
    def positive(cls, plane, point) -> bool:
        raise
    @classmethod
    def classify(cls, plane, face) -> int:
        raise
    @classmethod
    def split_volume(cls, vol, plane):
        return ([], [])
    @classmethod
    def is_adjacent(cls, vol, plane) -> bool:
        return False
    def __init__(self, faces, volume):
        self.plane = None
        self.faces = faces
        self.volume = volume
        self.child_f = None
        self.child_b = None

        self.sides = []

    def split(self, plane):
        if len(self.faces) == 0:
            return
        self.plane = plane
        f = []
        b = []
        inc = []
        for i in self.faces:
            r = self.__class__.classify(plane, i)
            if r == 0:
                inc.append(i)
            else:
                if r & 0x1:
                    f.append(i)
                if r & 0x2:
                    b.append(i)
        
        fv, bv = self.__class__.split_volume(self.volume, plane)
        self.child_f = _bsp_sub(f, fv)
        self.child_b = _bsp_sub(b, bv)
        self.volume = None
        self.faces = inc
        yield self.child_f
        yield self.child_b

    def find(self, point):
        c = self.__class__.positive(self.plane, point)
        if c:
            if self.child_f is not None:
                return self.child_f.find(point)
            return self
        if self.child_b is not None:
            return self.child_b.find(point)
        return self

    

class bsp_sub_3d(_bsp_sub):
    @classmethod
    def positive(cls, plane, point) -> bool:
        return meshutils.point_positive(plane, point)
    @classmethod
    def classify(cls, plane, face) -> int:
        return meshutils.classify_face(plane, face)
    @classmethod
    def split_volume(cls, vol, plane):
        return meshutils.split(vol, plane)

class bsp_tree:
    def __init__(self, mesh, flags: bsp_content_flags, mode='SUBTRACTIVE'):
        self.mode = mode
        self.mesh = bmesh.new()
        self.mesh.from_mesh(mesh)
        self.flags = flags
        self._build_flags_layer = self.mesh.faces.layers.int("face_build_flags")
        self._flags_layer = self.mesh.faces.layers.int("face_flags")


    def _build_tree(self):
        if len(self.faces) == 0:
            return None
        self.volume = box.bbox(self.mesh).create_box(m, 16)
        self.faces = list(self.mesh.faces[:])
        self.plane = None
        # find best splitter

    
    def _pick_splitter(self, faces):
        hint = None
        areaportal = None
        for i in self.faces:
            if self._get_build_flag(i, bsp_face_build_flags.hint):
                hint = i
                break
            if not areaportal and self._get_build_flag(i, bsp_face_build_flags.hint):
                areaportal = i
        if not hint:
            hint = areaportal if areaportal else self.faces[0]
        p = meshutils.get_plane(hint)



    def _get_build_flag(self, face, flag):
        pass

In [3]:
m = bmesh.new()

In [4]:
bmesh.ops.create_cube(m, size=10, matrix=bm.Matrix.LocRotScale(bm.Vector((1, 0, 0)), None, None))
[x.co for x in m.verts]

[Vector((-4.0, -5.0, -5.0)),
 Vector((-4.0, -5.0, 5.0)),
 Vector((-4.0, 5.0, -5.0)),
 Vector((-4.0, 5.0, 5.0)),
 Vector((6.0, -5.0, -5.0)),
 Vector((6.0, -5.0, 5.0)),
 Vector((6.0, 5.0, -5.0)),
 Vector((6.0, 5.0, 5.0))]

In [6]:
m.faces.ensure_lookup_table()

In [81]:
bb = box.bbox(m)

In [82]:
bb.radius()

17.320508075688775

In [42]:
u = bmesh.new()
bb.create_box(u, 5)
u.verts.ensure_lookup_table()
[x.co for x in u.verts[:]]

<Matrix 4x4 (20.0000,  0.0000,  0.0000, 1.0000)
            ( 0.0000, 20.0000,  0.0000, 0.0000)
            ( 0.0000,  0.0000, 20.0000, 0.0000)
            ( 0.0000,  0.0000,  0.0000, 1.0000)>


[Vector((-9.0, -10.0, -10.0)),
 Vector((-9.0, -10.0, 10.0)),
 Vector((-9.0, 10.0, -10.0)),
 Vector((-9.0, 10.0, 10.0)),
 Vector((11.0, -10.0, -10.0)),
 Vector((11.0, -10.0, 10.0)),
 Vector((11.0, 10.0, -10.0)),
 Vector((11.0, 10.0, 10.0))]

In [28]:
flags_layer = m.faces.layers.int.new("flags")

In [7]:
f = m.faces[0]

In [32]:
f[flags_layer] = 123

In [13]:
bm.Vector((*f.normal, f.verts[0].co @ f.normal))

Vector((-1.0, -0.0, 0.0, 4.0))

In [33]:
m0 = bpy.data.meshes.new("m0")

In [38]:
o0 = bpy.data.objects.new("o0", object_data=m0)

In [40]:
m.to_mesh(m0)

In [44]:
p0 = o0.data.polygons[0]

In [49]:
mm = bmesh.new()
mm.from_mesh(m0)

In [60]:
mm.faces[0][mm.faces.layers.int["flags"]]

123

In [67]:
v = m.verts[0].co

In [72]:
[x.co for x in m.verts[:]]

[Vector((-4.0, -5.0, -5.0)),
 Vector((-4.0, -5.0, 5.0)),
 Vector((-4.0, 5.0, -5.0)),
 Vector((-4.0, 5.0, 5.0)),
 Vector((6.0, -5.0, -5.0)),
 Vector((6.0, -5.0, 5.0)),
 Vector((6.0, 5.0, -5.0)),
 Vector((6.0, 5.0, 5.0)),
 Vector((-4.0, -1.0, 5.0)),
 Vector((-4.0, 5.0, -1.0)),
 Vector((0.0, 5.0, -5.0)),
 Vector((6.0, -1.0, -5.0)),
 Vector((6.0, -5.0, -1.0)),
 Vector((0.0, -5.0, 5.0))]

In [71]:
plane = bm.Vector((1.0, 1.0, 1.0, 0.0))
bmesh.ops.bisect_plane(m, geom=m.faces[:] + m.edges[:] + m.verts[:], 
            plane_co=plane.xyz * plane.w, plane_no=plane.xyz)

{'geom_cut': [<BMVert(0x5646f6fab6f0), index=8>,
  <BMVert(0x5646f6fab728), index=9>,
  <BMVert(0x5646f6fab760), index=10>,
  <BMVert(0x5646f6fab798), index=11>,
  <BMVert(0x5646f6fab7d0), index=12>,
  <BMVert(0x5646f6fab808), index=13>,
  <BMEdge(0x5646f6fb3ab0), index=18, verts=(0x5646f6fab7d0/12, 0x5646f6fab808/13)>,
  <BMEdge(0x5646f6fb3b00), index=19, verts=(0x5646f6fab7d0/12, 0x5646f6fab798/11)>,
  <BMEdge(0x5646f6fb3b50), index=20, verts=(0x5646f6fab760/10, 0x5646f6fab798/11)>,
  <BMEdge(0x5646f6fb3ba0), index=21, verts=(0x5646f6fab728/9, 0x5646f6fab760/10)>,
  <BMEdge(0x5646f6fb3bf0), index=22, verts=(0x5646f6fab6f0/8, 0x5646f6fab728/9)>,
  <BMEdge(0x5646f6fb3c40), index=23, verts=(0x5646f6fab6f0/8, 0x5646f6fab808/13)>],
 'geom': [<BMVert(0x5646f6fab530), index=0>,
  <BMVert(0x5646f6fab568), index=1>,
  <BMVert(0x5646f6fab5a0), index=2>,
  <BMVert(0x5646f6fab5d8), index=3>,
  <BMVert(0x5646f6fab610), index=4>,
  <BMVert(0x5646f6fab648), index=5>,
  <BMVert(0x5646f6fab680), inde